### Run all cells, the interactive plot is at the bottom

In [1]:
using Interact, StatsPlots, DataFrames, CSV, Dates;
include("plotting_functions.jl");
#WebIO.install_jupyter_nbextension() # might need to reload page afterwards in order to work

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11004353160010124317\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9835894400365268980\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11004353160010124317\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
Input_df = CSV.read("data/200124_datafile_all_details_right_timestamp.csv", DataFrame);

#### Choose the parameter combination to evaluate in the interactive plot (see results folder)

In [3]:
h_predict=8760;
h_control=8760;
h_start=1;
h_end=8760;
objective=1;
case=1;
costfactor=1.0;

In [4]:
Flow_df = CSV.read("results/results_$(h_predict)_$(h_control)_$(h_start)"*
    "-$(h_end)_$(objective)_$(case)_$(costfactor).csv", DataFrame);

In [5]:
first(Flow_df, 3)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,22.0,180.0,13.5,-0.0,0.0,-0.0,0.0,0.0
2,22.0,179.329,11.9555,0.0,0.0,0.0,0.0,0.0
3,22.0,178.658,10.3964,0.0,0.0,0.0,0.0,0.0


In [6]:
first(Input_df, 3)

,heating,heating_fan,hotwater,lights,lgappl,vent_fan,misc,electkwh
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.73669,0.044709,0.0,0.022749,0.042381,0.0,0.176259,0.241389
2,3.83768,0.035999,0.0,0.013649,0.041584,0.0,0.166187,0.22142
3,6.05949,0.041142,0.0,0.009099,0.036289,0.0,0.161151,0.206539


In [7]:
# merge on data stamp
Data_df = leftjoin(Flow_df, Input_df, on=[:month, :day, :hour], makeunique=false, 
    indicator=nothing, validate=(false, false));
first(Data_df,3)

,Temp_FH,Vol_HW,Soc_B,V_HW_plus,V_HW_minus,T_FH_plus,T_FH_minus,profits
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,22.0,180.0,13.5,-0.0,0.0,-0.0,0.0,0.0
2,22.0,179.329,11.9555,0.0,0.0,0.0,0.0,0.0
3,22.0,178.658,10.3964,0.0,0.0,0.0,0.0,0.0


In [8]:
size(Data_df)

(8760, 36)

In [9]:
@manipulate for start_date=Date(2015,1,1):Day(1):Date(2015,12,31), 
    range_shown=1:1:8
    
    p1=bar_row(bar_PV, Data_df, start_date, range_shown);
    p2=bar_row(bar_demand, Data_df, start_date, range_shown);
    p3=bar_row(bar_heat, Data_df, start_date, range_shown);
    p4=bar_row(bar_comfort, Data_df, start_date, range_shown);
    
    vbox(vskip(1em),
        md"*Interactive illustration of SHEMS results:*",
        vskip(1em),
        plot(p1,p2,p3,p4, layout=(4,1), size=(range_shown*300,4*200)))
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["start_date"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 365,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
183, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\Lissy\\.julia\\packages\\Knockout\\IP1uR\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\InteractBase\\sOe2Z\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\Lissy\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000034c8bc70, Task (runnable) @0x0000000034c8bc70), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2015-01-01\",\"2015-01-02\",\"2015-01-03\",\"2015-01-04\",\"2015-01-05\",\"2015-01-06\",\"2015-01-07\",\"2015-01-08\",\"2015-01-09\",\"2015-01-10\",\"2015-01-11\",\"2015-01-12\"